In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
import os
import cv2
from numpy import asarray
from PIL import Image
from functools import partial
from keras.applications import imagenet_utils

Using TensorFlow backend.


In [2]:
epochs = 10
batch_size = 16

In [3]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

validation_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        batch_size=batch_size,
		directory='datasets/compiled/train/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        batch_size=batch_size,
        directory='datasets/compiled/validation/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

Found 2452 images belonging to 2 classes.
Found 613 images belonging to 2 classes.


In [5]:
### Pre-trained Model (DenseNet121 trained on Imagenet)
model = tf.keras.applications.DenseNet121(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# Transfer Learning
for i in model.layers:
    i.trainable = False

In [7]:
global_avg = tf.keras.layers.GlobalAveragePooling2D()(model.output)
flatten = tf.keras.layers.Flatten()(global_avg)
out = tf.keras.layers.Dense(2,activation='softmax')(flatten)
densenet = tf.keras.Model(inputs=[model.input],outputs=[out])

In [8]:
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=["accuracy"])

In [9]:
history = densenet.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 92/154 [================>.............] - ETA: 28s - loss: 0.4375 - acc: 0.8473

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\PIL\Image.py:963: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


154/154 [==============================] - 88s 571ms/step - loss: 0.3228 - acc: 0.8874 - val_loss: 0.1042 - val_acc: 0.9592
Epoch 2/10
154/154 [==============================] - 65s 421ms/step - loss: 0.1590 - acc: 0.9392 - val_loss: 0.3235 - val_acc: 0.9152
Epoch 3/10
154/154 [==============================] - 65s 419ms/step - loss: 0.2584 - acc: 0.9274 - val_loss: 0.1750 - val_acc: 0.9511
Epoch 4/10
154/154 [==============================] - 64s 413ms/step - loss: 0.1593 - acc: 0.9482 - val_loss: 0.1946 - val_acc: 0.9413
Epoch 5/10
154/154 [==============================] - 50s 325ms/step - loss: 0.1669 - acc: 0.9494 - val_loss: 0.0794 - val_acc: 0.9804
Epoch 6/10
154/154 [==============================] - 41s 266ms/step - loss: 0.1959 - acc: 0.9470 - val_loss: 0.6342 - val_acc: 0.8695
Epoch 7/10
154/154 [==============================] - 41s 264ms/step - loss: 0.1881 - acc: 0.9584 - val_loss: 1.6080 - val_acc: 0.7504
Epoch 8/10
154/154 [==============================] - 42s 272ms/st

In [12]:
densenet.save('densenet121_detection_model.h5')

In [10]:
### Metrics
# Testing directories
test_mask_imgs = os.listdir('datasets/compiled/test/mask')
test_no_mask_imgs = os.listdir('datasets/compiled/test/no-mask')

# Numbers of images
total_mask = len(test_mask_imgs)
total_no_mask = len(test_no_mask_imgs)
total = total_mask + total_no_mask

# Confusion Matrix variables
mask_correct = 0
mask_incorrect = 0
no_mask_correct = 0
no_mask_incorrect = 0

# Mask class
for img in test_mask_imgs:
    
    tmp = Image.open('datasets/compiled/test/mask/'+ img)
    test_img = np.asarray(tmp)
    test = test_img.copy()
    test.resize(1,224,224,3)
    ans = densenet.predict(test)
    print(ans)
    if ans[0][1] > ans[0][0]:
        mask_correct += 1
        print("mask")
    elif ans[0][1] < ans[0][0]:
        mask_incorrect += 1
        print("no-mask")
        
    # Mask recall
    mask_recall = mask_correct/total_mask

# No-mask class
for img in test_no_mask_imgs:
    
    tmp = Image.open('datasets/compiled/test/no-mask/' + img)
    test_img = np.asarray(tmp)
    test = test_img.copy()
    test.resize(1,224,224,3)
    ans = densenet.predict(test)
    print(ans)
    
    if ans[0][0] > ans[0][1]:
        no_mask_correct += 1
        print("no-mask")
    elif ans[0][0] < ans[0][1]:
        no_mask_incorrect += 1
        print("mask")
        
    # No-mask recall
    no_mask_recall = no_mask_correct/total_no_mask

# Mask precision
mask_precision = (mask_correct)/(mask_correct + mask_incorrect)

# No-mask precision
no_mask_precision = (no_mask_correct)/(no_mask_correct + no_mask_incorrect)

# Mask F1 score
mask_f1 = 2 * ((mask_recall * mask_precision)/(mask_recall + mask_precision))

# No-mask F1 score
no_mask_f1 = 2 * ((no_mask_recall * no_mask_precision)/(no_mask_recall + no_mask_precision))

# Classification accuracy
accuracy = (mask_correct + no_mask_correct)/total

# Weighted recall
recall = ((mask_recall*total_mask)+(no_mask_recall*total_no_mask))/(total_mask+total_no_mask)

# Weight precision
precision = ((mask_precision*total_mask)+(no_mask_precision*total_no_mask))/(total_mask+total_no_mask)

# Weighted F1 score
f1 = 2 * ((recall * precision)/(recall + precision))

[[0. 1.]]
mask
[[1.0000000e+00 1.4881207e-08]]
no-mask
[[0. 1.]]
mask
[[2.6370548e-13 1.0000000e+00]]
mask
[[0.994005 0.005995]]
no-mask
[[2.3613388e-25 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[2.5014407e-10 1.0000000e+00]]
mask
[[8.438749e-10 1.000000e+00]]
mask
[[0. 1.]]
mask
[[5.0948384e-21 1.0000000e+00]]
mask
[[2.3631895e-11 1.0000000e+00]]
mask
[[2.4940657e-29 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[4.6504117e-32 1.0000000e+00]]
mask
[[1.805849e-36 1.000000e+00]]
mask
[[0. 1.]]
mask
[[0.05085086 0.9491492 ]]
mask
[[3.4160445e-11 1.0000000e+00]]
mask
[[1.2398948e-08 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[0.99487513 0.00512484]]
no-mask
[[1.4930164e-38 1.0000000e+00]]
mask
[[3.5334244e-16 1.0000000e+00]]
mask
[[5.5916534e-08 1.0000000e+00]]
mask
[[2.4327721e-13 1.0000000e+00]]
mask
[[2.5887586e-32 1.0000000e+00]]
mask
[[7.7408396e-07 9.9999928e-01]]
mask
[[1.0757136e-12 1.0000000e+00]]
mask
[[0.98999715 0.01000293]]
no-mask
[[4.2630435e-07 9.9999952e-01]]
mask
[[0. 1.]]
mask
[[1.1495355e

[[1.4930377e-05 9.9998510e-01]]
mask
[[0. 1.]]
mask
[[1.000000e+00 1.212287e-15]]
no-mask
[[1.0000000e+00 2.7899492e-16]]
no-mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[1.000000e+00 1.212287e-15]]
no-mask
[[1.0000000e+00 1.1249222e-15]]
no-mask
[[1.000000e+00 1.212287e-15]]
no-mask
[[1.000000e+00 1.212287e-15]]
no-mask
[[1.000000e+00 1.212287e-15]]
no-mask
[[1.3436206e-38 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[1.000000e+00 1.212287e-15]]
no-mask
[[1.0000000e+00 1.1329459e-14]]
no-mask
[[9.637342e-20 1.000000e+00]]
mask
[[1.000000e+00 1.212287e-15]]
no-mask
[[0. 1.]]
mask
[[1.0000000e+00 3.4755473e-12]]
no-mask
[[1.0000000e+00 2.8673239e-11]]
no-mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[3.6304836e-26 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[3.6627748e-31 1.0000000e+00]]
mask
[[1.0000000e+00 1.9114477e-31]]
no-mask
[[3.071836e-13 1.000000e+00]]
mask
[[5.2133367e-34 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[8.313672e-14 1.000000e+00]]
mask
[[0. 1.]]
mask
[[1.000000e+00 1.212287e-15]]
no-mask
[

[[0. 1.]]
mask
[[3.759259e-24 1.000000e+00]]
mask
[[7.855502e-38 1.000000e+00]]
mask
[[1.3008882e-13 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[6.694226e-37 1.000000e+00]]
mask
[[0. 1.]]
mask
[[9.769009e-26 1.000000e+00]]
mask
[[0. 1.]]
mask
[[2.2091703e-24 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[1.7322373e-05 9.9998271e-01]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[5.052044e-15 1.000000e+00]]
mask
[[3.7408514e-37 1.0000000e+00]]
mask
[[8.813638e-37 1.000000e+00]]
mask
[[2.5127257e-29 1.0000000e+00]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[6.1725485e-05 9.9993825e-01]]
mask
[[1.863613e-29 1.000000e+00]]
mask
[[9.9980062e-01 1.9942941e-04]]
no-mask
[[0. 1.]]
mask
[[3.9877025e-18 1.0000000e+00]]
mask
[[2.9828613e-17 1.0000000e+00]]
mask
[[1.8403245e-18 1.0000000e+00]]
mask
[[9.9981099e-01 1.8905831e-04]]
no-mask
[[0. 1.]]
mask
[[0. 1.]]
mask
[[7.6722217e-28 1.0000000e+00]]
mask
[[1.8971014e-12 1.0000000e+00]]
mas

In [11]:
print("Mask correct predictions: {}".format(mask_correct))
print("Mask incorrect predictions: {}".format(mask_incorrect))

print("No-mask correct predictions: {}".format(no_mask_correct))
print("No-mask incorrect predictions: {}".format(no_mask_incorrect))

print("Mask total images: {}".format(total_mask))
print("No-mask total images: {}".format(total_no_mask))

print("Total test images: {}".format(total))

print("Mask recall: {}".format(mask_recall))
print("No-mask recall: {}".format(no_mask_recall))

print("Mask precision: {}".format(mask_precision))
print("No-mask precision: {}".format(no_mask_precision))

print("Mask F1 score: {}".format(mask_f1))
print("No-mask F1 score: {}".format(no_mask_f1))

print("Classification accuracy: {}".format(accuracy))
print("Weighted recall: {}".format(recall))
print("Weighted precision: {}".format(precision))
print("Weighted F1 score: {}".format(f1))

Mask correct predictions: 330
Mask incorrect predictions: 54
No-mask correct predictions: 19
No-mask incorrect predictions: 365
Mask total images: 384
No-mask total images: 384
Total test images: 768
Mask recall: 0.859375
No-mask recall: 0.049479166666666664
Mask precision: 0.859375
No-mask precision: 0.049479166666666664
Mask F1 score: 0.859375
No-mask F1 score: 0.049479166666666664
Classification accuracy: 0.4544270833333333
Weighted recall: 0.4544270833333333
Weighted precision: 0.4544270833333333
Weighted F1 score: 0.4544270833333333
